In [ ]:
# | default_exp wb_veg

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
# | export

import warnings
import collections
import numpy as np
from typing import Dict
from pandera.typing import DataFrame
from sureau_ecos_py.plant_utils import plc_comp

from sureau_ecos_py.create_vegetation_parameters import (
    create_vegetation_parameters
)
from sureau_ecos_py.create_climate_data import create_climate_data
from sureau_ecos_py.create_modeling_options import create_modeling_options
from sureau_ecos_py.create_soil_parameters import create_soil_parameters
from sureau_ecos_py.create_stand_parameters import create_stand_parameters

In [ ]:
# | export
def new_wb_veg(veg_params:Dict # Dictionary created using the `create_vegetation_parameters` functions
) -> Dict:

    "Create an object wb_veg from veg_params"

    # Assert parameters ---------------------------------------------------------
    assert (
        isinstance(veg_params, Dict)
    ), f"veg_param must be a Dictionary not a {type(veg_params)}"


    # Add params to wb_veg dictionary -------------------------------------------

    # Create Empty dict
    wb_veg = collections.defaultdict(list)

    # Add veg_params
    wb_veg["params"] = veg_params

    # Add Stem and Leaf turgor point
    wb_veg['params']['psi_tlp_leaf'] = wb_veg["params"]["pifullturgor_leaf"]*wb_veg["params"]["epsilonsym_leaf"]/(wb_veg["params"]["pifullturgor_leaf"]+wb_veg["params"]["epsilonsym_leaf"])
    wb_veg['params']['psi_tlp_stem'] = wb_veg["params"]["pifullturgor_stem"]*wb_veg["params"]["epsilonsym_stem"]/(wb_veg["params"]["pifullturgor_stem"]+wb_veg["params"]["epsilonsym_stem"])

    # Add potentials
    wb_veg['psi_lapo'] = 0
    wb_veg['psi_sapo'] = 0
    wb_veg['psi_lsym'] = 0
    wb_veg['psi_ssym'] = 0

    # FP replaced "mem" by "cav" (when cavitation starts)
    wb_veg['psi_lapo_cav'] = 0
    wb_veg['psi_sapo_cav'] = 0
    wb_veg['psi_all_soil'] = 0

    # Conductance & capacitance (mmol/m2/s/MPa) Here on leaf area basis but they
    # are to be updated as a function symplasm conductance and leaf area hydraulic
    # conductances

    # constant value during simulation
    wb_veg["k_plant"] =  wb_veg["params"]["k_plant_init"]

    # constant value during simulation
    wb_veg["k_lsym"]  =  wb_veg["params"]["k_lsym_init"]

    # constant value during simulation
    wb_veg["k_ssym"]  =  wb_veg["params"]["k_ssyminit"]

    # Value is updated in compute_kplant_wb_veg
    wb_veg["k_rsapo"] =  np.nan

    # Value is updated in compute_kplant_wb_veg
    wb_veg["k_slapo"] =  np.nan

    # Conductance rhisophere for each soil layer
    wb_veg['k_soil_to_stem'] = np.array([np.nan, np.nan, np.nan], dtype=float)

    # Capacitances
    # Symplasm capacitance updated as a function of water content and Leaf area
    # (mmol/m2leaf/MPa) /updated in update.capacitancesSymAndApo()
    # (NM : 25/10/2021)

    wb_veg['c_lsym'] = np.nan
    wb_veg['c_sapo'] = np.nan

    # Leaf and canopy conductance

    # initialised at 0 to compute Tleaf on first time step considering gs = 0
    # and not NA s
    wb_veg['gmin'] = 0

    # Gmin for stem and branches
    wb_veg['gmin_s'] = wb_veg["params"]["gmin_s"]

    # TODO voir si y'a besoin d'initialiser
    # see if there is a need to initialize

    wb_veg['regul_fact'] = 0.01

    # TODO voir pour mettre tout en NA si TranspirationMod = 0
    # see to put everything in NA if TranspirationMod = 0

    wb_veg['gs_bound'] = np.nan

    # initialised to 0 to compute Tleaf on first time step considering gs = 0
    # and not NA
    wb_veg['gs_lim'] = 0
    wb_veg['gcanopy_bound'] = np.nan
    wb_veg['gcanopy_lim'] = np.nan
    wb_veg['g_bl'] = np.nan
    wb_veg['g_crown'] = np.nan

    # Fluxes
    wb_veg['e_prime'] = 0
    wb_veg['e_min'] = 0
    wb_veg['e_min_s'] = 0
    wb_veg['e_bound'] = 0
    wb_veg['e_lim'] = 0
    wb_veg['flux_soil_to_stem'] = np.zeros(3)
    wb_veg['transpiration_mm'] = 0
    wb_veg['e_min_mm'] = 0
    wb_veg['e_min_s_mm'] = 0

    # LAI and LAI-dependent variables
    wb_veg['lai_pheno'] = np.zeros(1)
    wb_veg['lai']      = np.zeros(1)
    wb_veg['canopy_storage_capacity'] = np.zeros(1)

    # rainfall and interception

    # ppt that reach the soil
    wb_veg['ppt_soil'] = 0

    # interceptedWater /quantite d'eau dans la canopee
    wb_veg['intercepted_water_amount'] = 0
    wb_veg['evaporation_intercepted'] = 0
    wb_veg['etpr'] = 0

    # defoliation // no defoliation (add an option to set defoliation due to
    # cavitation of the Plant Above)
    wb_veg['defoliation'] = 0
    wb_veg['lai_dead'] = 0


    # Cavitation
    # percent loss of conductivity [%]
    wb_veg["plc_leaf"] = 0

    # percent loss of conductivity [%]
    wb_veg["plc_stem"] = 0

    # leaf temp
    wb_veg["leaf_temperature"] = np.nan

    # Pheno, parameters if deciduous
    if wb_veg["params"]["foliage"] == "deciduous":

        wb_veg['lai_pheno'] = 0

        # temmpeerature sum to determine budburst
        wb_veg['sum_temperature'] = 0

        # budburst date
        wb_veg['bud_burst_date'] = np.nan

        print("wb_veg params for deciduous forest created")

    # water storage in canopy

    # Fuel moisture content of the dead compartment (gH20/gMS)
    wb_veg['dfmc']     = np.nan

    # Live fuel moisture content of the apoplasmic compartment (gH20/gMS)
    wb_veg['lfmc_apo']  = np.nan

    # Live fuel moisture content of the apoplasmic compartment (gH20/gMS)
    wb_veg['lfmc_symp'] = np.nan

    # live fuel moisture content (gH20/gMS)
    wb_veg['lfmc']     = np.nan

    # Live Canopy dry matter [gMS/m2 soil]
    wb_veg['dm_live_canopy']  = np.nan

    # Dead Canopy dry matter [gMS/m2 soil]
    wb_veg['dm_dead_canopy']  = 0

    # Q leaf apo (mol/m2leaf)
    wb_veg['q_lapo_sat_mmol'] = 0
    wb_veg['q_lapo_sat_l'] = 0

    # Q stem apo (mol/m2leaf)
    wb_veg['q_sapo_sat_mmol'] = 0
    wb_veg['q_sapo_sat_l'] = 0

    # Q leaf symplasm (mol/m2leaf)
    wb_veg['q_lsym_sat_mmol'] = 0
    wb_veg['q_lsym_sat_l']    = 0

    # Q Stem symplasm (mol/m2leaf)
    wb_veg['q_ssym_sat_mmol'] = 0
    wb_veg['q_ssym_sat_l']    = 0

    # Q Stem and Leaf apo and symp in liter/kg TODO 13/08/2021: better in mmol?
    wb_veg['q_lapo_l'] = 0
    wb_veg['q_sapo_l'] = 0
    wb_veg['q_lsym_l'] = 0
    wb_veg['q_ssym_l'] = 0

    wb_veg['delta_q_lapo_mmol_diag'] = 0

    wb_veg['f_l_cav'] = 0
    wb_veg['f_s_cav'] = 0

    # Compute PLC
    wb_veg['plc_leaf'] = plc_comp(psi = wb_veg['psi_lapo'],
                                  slope = wb_veg['params']['slope_vc_leaf'],
                                  p50 = wb_veg['params']['p50_vc_leaf']
                                  )

    #wb_veg['plc_stem'] = plc_comp(psi = WBveg$Psi_SApo,
    #                              slope = WBveg$params$slope_VC_Stem,
    #                              P50 = WBveg$params$P50_VC_Stem)


    return wb_veg


#### __Example: __

In [ ]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="sigmoid",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="campbell",
)

In [ ]:
stand_params = create_stand_parameters(
    file_path="./sample_data/stand_example.csv",
    lai_max=None,
    latitude=None,
    longitude=None,
    sep=";",
)

In [ ]:
soil_params = create_soil_parameters(
    file_path="./sample_data/soil_example.csv",
    modeling_options=modeling_options_dict,
    default_soil=False,
    offset_psoil=1,
    psoil_at_field_capacity=-1,
)

There is an offset on Psoil of 1 MPa
Psoil at field capacity = -0.001 MPa
You are using campbell pedotransfer formulation
Available water capacity Wilting: 273.74477742718403 mm
Available water capacity Residual: 307.81195119276424 mm
Can soil_params["v_soil_storage_capacity"] be negative?? Ask


In [ ]:
vegetation_params = create_vegetation_parameters(
    file_path="./sample_data/vegetation_example_deciduous_beech_wide.csv",
    list_of_parameters=None,
    soil_parameters=soil_params,
    stand_parameters=stand_params,
    modeling_options=modeling_options_dict,
    sep=",",
)

frac_leaf_sym' set to 0.4
Available water capacity @Tlp (Campbell):278.27401951129264 mm
Available water capacity @P50 (Campbell):282.61737155892024 mm


/workspaces/sureau_ecos_py/sureau_ecos_py/create_vegetation_parameters.py:106: UserWarning: Make sure that depth from soil_parameters is in meters
  warnings.warn("Make sure that depth from soil_parameters is in meters")


In [ ]:
new_wb_veg(vegetation_params)

wb_veg params for deciduous forest created


TypeError: plc_comp() got an unexpected keyword argument 'psi'